**Garmin Connect**

In [40]:
import pandas as pd
import glob
from datetime import datetime

In [ ]:
dfs = []
for path in file_paths:
    df = pd.read_csv(path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.head()

In [ ]:
# Selecting only the first 2 columns of the DataFrame:

df = df.iloc[:, :4]
df.head(10)

In [ ]:
# Converting the DataFrame to a list, merging consecutive rows into a single row, and converting the merged data back to a DataFrame

data_values = df.values.tolist()
merged_data = [data_values[i] + data_values[i+1] for i in range(0, len(data_values)-1)]
df_merged = pd.DataFrame(merged_data, columns=list(df.columns)*2)
display(df_merged)

In [ ]:
# Renaming the "Czas" column to "Czas2", removing rows that contain a colon in the "Czas" column, and selecting only 5 columns of the DataFrame

index = df_merged.columns.tolist().index('Czas', 1)
df_merged.columns.values[index] = 'Czas2'
df_merged = df_merged[~df_merged['Czas'].str.contains(':')]
df_merged = df_merged.iloc[:, [0,4,5,6,7]]
display(df_merged)

In [ ]:
def replace_month(date_str):
    months_pl_to_en = {
        'Sty': 'Jan', 'Lut': 'Feb', 'Mar': 'Mar', 'Kwi': 'Apr',
        'Maj': 'May', 'Cze': 'Jun', 'Lip': 'Jul', 'Sie': 'Aug',
        'Wrz': 'Sep', 'Paź': 'Oct', 'Lis': 'Nov', 'Gru': 'Dec'
    }
    for pl, en in months_pl_to_en.items():
        date_str = date_str.replace(pl, en)
    return date_str.strip()

df_merged['Czas'] = df_merged['Czas'].apply(replace_month)
df_merged['Czas'] = pd.to_datetime(df_merged['Czas'], format="%d %b %Y").dt.strftime('%Y-%m-%d')

df_merged

In [ ]:
df_merged = df_merged.rename(columns={'Czas': 'Date', 'Czas2' : 'Time', 'Ciężar': 'Weight'})
df_merged = df_merged.reindex(columns=['Date', 'Time', 'Weight'])
df_merged['Weight'] = df_merged['Weight'].str.replace(' kg', '').astype(float)
display(df_merged)

In [ ]:
duplicated_rows = df_merged.duplicated()

print(f"Number of duplicated rows: {duplicated_rows.sum()}")

if duplicated_rows.sum() > 0:
  df_merged = df_merged.drop_duplicates()
  print (f"Deleted {duplicated_rows.sum()} rows")

df_merged = df_merged.sort_values(by=['Date', 'Time'], ascending=True)

display(df_merged)

In [50]:
print(f'Number of entries: {len(df_merged)}')
print(f'Date of first entry: {df_merged["Date"].iloc[0]}')
print(f'Date of last entry: {df_merged["Date"].iloc[-1]}')

Number of entries: 378
Date of first entry: 2020-12-06
Date of last entry: 2024-03-01


In [ ]:
# Export the merged dataframe to the weight.csv file
df_merged.to_csv('/content/drive/MyDrive/Colab Notebooks/Tableau/Outputs/weight.csv', index=False)